In [ ]:
from ipywidgets import IntSlider, Dropdown, interactive
from IPython.display import display
import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np
import os

In [ ]:
def _display_slices(slices: list, model_name: str):
    """
    Function to plot a slice of an MRI image

    Args:
        slices: List of 2D slices to plot
        model_name: Name of the model for labeling

    Returns:
        None
    """

    # Normalize slices to [0, 1] for proper blending
    slices = [
        np.clip(slice_data / np.max(slice_data), 0, 1) if np.max(slice_data) > 0 else slice_data
        for slice_data in slices
    ]

    # Separate the images
    original = slices[0]
    ground_truth = slices[1]
    prediction = slices[2]

    # Create a new RGB image for overlay
    overlay = np.zeros((*original.shape, 3))  
    overlay[..., 0] = prediction  
    overlay[..., 2] = ground_truth 

    overlay_truth = np.zeros((*original.shape, 3)) 
    overlay_truth[..., 2] = ground_truth 

    overlay_pred = np.zeros((*original.shape, 3))  
    overlay_pred[..., 0] = prediction  

    # Plot the images
    _, axes = plt.subplots(1, 4, figsize=(20, 20))

    title_kwargs = {'fontsize': 16, 'fontweight': 'bold'}  # Custom title formatting

    axes[0].imshow(original, cmap='gray')
    axes[0].set_title("Corte Original", **title_kwargs)
    axes[0].axis('off')

    axes[1].imshow(original, cmap='gray')  
    axes[1].imshow(overlay_truth, alpha=0.5)  
    axes[1].set_title("Máscara de Segmentación", **title_kwargs)
    axes[1].axis('off')

    axes[2].imshow(original, cmap='gray')  
    axes[2].imshow(overlay_pred, alpha=0.5)  
    axes[2].set_title(f"Prediction de {model_name}", **title_kwargs)
    axes[2].axis('off')

    axes[3].imshow(original, cmap='gray')  
    axes[3].imshow(overlay, alpha=0.5)  
    axes[3].set_title("Superposición", **title_kwargs)
    axes[3].axis('off')

    plt.tight_layout()
    plt.show()

In [ ]:
def display_slices(
    slice_type: str,
    slice_index: int,
    nifti_files: list,
    model_name: str = "nnUNet2D"
):
    """
    slice_type: 'Axial' | 'Coronal' | 'Sagittal'
    slice_index: índice del corte
    nifti_files: [original.nii.gz, ground_truth.nii.gz, prediction.nii.gz]
    add_fp, add_fn, remove_tp: fracciones [0,1] para simular FP, FN y eliminar TP.
    seed: semilla para reproducibilidad.
    """
    
    # 1) Cargar volúmenes
    orig_vol, gt_vol, pred_vol = [nib.load(f).get_fdata() for f in nifti_files]

    # 2) Extraer el corte 2D de cada volumen
    def get_slice(vol):
        if slice_type == "Axial":
            return vol[:, :, slice_index]
        elif slice_type == "Coronal":
            return vol[:, slice_index, :]
        elif slice_type == "Sagittal":
            return vol[slice_index, :, :]
        else:
            raise ValueError(f"slice_type inválido: {slice_type}")

    slices = [
        get_slice(orig_vol),
        get_slice(gt_vol),
        get_slice(pred_vol)
    ]

    # 3) Mostrar como hasta ahora
    _display_slices(slices, model_name)

In [ ]:
def display_slices_interactive(nifti_files: list, 
                               model_name: str = "nnUNet3D", 
                               ):
    # Create a dropdown for configuration
    config_dropdown = Dropdown(
        options=["Axial", "Coronal", "Sagittal"],
        value="Sagittal",
        description='Orientation:'
    )

    # Create a slider for the slice index
    slice_slider = IntSlider(value=111, min=0, max=181, step=1, description="Slice Index")

    # Function to update the slider's max value based on the dropdown
    def update_slider(change):
        if change['new'] == "Coronal":
            slice_slider.max = 217
        else:
            slice_slider.max = 181

    # Link the dropdown to update the slider
    config_dropdown.observe(update_slider, names='value')

    # Wrapper function to fix 'data'
    def wrapped_display_slices(slice_type, slice_index):
        display_slices(slice_type=slice_type, 
                       slice_index=slice_index, 
                       nifti_files=nifti_files, 
                       model_name=model_name)

    # Create an interactive widget
    interactive_widget = interactive(wrapped_display_slices, slice_type=config_dropdown, slice_index=slice_slider)

    # Display the interactive widget
    display(interactive_widget)

In [ ]:
cwd = os.getcwd()

nnUNet_raw = f"{cwd}/validation"
flair = f"{nnUNet_raw}/original/BRATS_86.nii"
mask = f"{nnUNet_raw}/mask/BRATS_86.nii"
nnUNet_prediction = f"{nnUNet_raw}/nnUNet3D/BRATS_86.nii.gz"

nifti_files = [flair, mask, nnUNet_prediction]

display_slices_interactive(nifti_files, 
                           model_name="YoloD-Sagittal")

interactive(children=(Dropdown(description='Orientation:', options=('Axial', 'Coronal', 'Sagittal'), value='Ax…